In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
# Importing important libraries

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

2026-02-08 06:31:36.293121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770532296.564968      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770532296.646336      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770532297.287808      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770532297.287870      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770532297.287877      55 computation_placer.cc:177] computation placer alr

## 1. Importing Data

In [3]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

* data is balanced

In [6]:
### Map the sentiment values to numeric
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)

/tmp/ipykernel_55/3525237038.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)


In [7]:
df['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

## Text Preprocessing

#### Step 1. Lowercasing 

In [8]:
df['review'] = df['review'].str.lower()
df['review'].head(2)

0    one of the other reviewers has mentioned that ...
1    a wonderful little production. <br /><br />the...
Name: review, dtype: object

#### Step 2. Remove HTML Tags 

In [9]:
import re

def remove_html_tags(text):
    clean_text = re.sub('<.*?>', '',text)
    return clean_text

In [10]:
df['review'] = df['review'].apply(remove_html_tags)
df['review'].head(2)

0    one of the other reviewers has mentioned that ...
1    a wonderful little production. the filming tec...
Name: review, dtype: object

#### Step 3. Remove Punctuations

In [11]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
exclude = string.punctuation

def remove_punctuation(text):
    for char in exclude:
        text = text.replace(char, '')
        return text

In [13]:
df['review'] = df['review'].apply(remove_punctuation)

#### Step 4. Removing Stopwords

In [14]:
from nltk.corpus import stopwords

In [15]:
# It has all stopwords
eng_stopwords = stopwords.words('english')

# Keeping all negative words explicitly
negation_words = {"not", "no", "never", "n't", "hardly", "barely", "isn’t", "is not",
                  "wasn’t", "was not", "don’t", "do not", "didn’t", "did not",
                  "wouldn't","would not", "not_good", "not good"}

# Defining function
def remove_stopword(text):
    text = text.lower()
    tokens = text.split()
    tokens = [w for w in tokens if w not in eng_stopwords or w in negation_words]
    return " ".join(tokens)

In [16]:
df['review'] = df['review'].apply(remove_stopword)

In [17]:
df['review'].head(2)

0    one reviewers mentioned watching 1 oz episode ...
1    wonderful little production. filming technique...
Name: review, dtype: object

#### Step 5. Negation Tagging

In [18]:
def handle_negation(text):
    text = re.sub(r"not\s+(\w+)", r"not_\1", text)
    return text

* How it works: [not good] --> [not_good]

In [19]:
df['review'] = df['review'].apply(handle_negation)

#### Step 6. Add ngram

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

def add_bigrams(text):
    words = text.split()
    bigrams = [words[i] + "_" + words[i+1] for i in range(len(words)-1)]
    return text + " " + " ".join(bigrams)

df["review"] = df["review"].apply(add_bigrams)

## 2. Train Test Split

In [21]:
from sklearn.model_selection import train_test_split

# X = Features, y = Target/Label
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'],
                                                    test_size=0.2, random_state=42)

### Adding short-review bias correction

Short reviews are rare in IMDB → model underfits them.

#### Synthetic short samples (simple & effective)

In [22]:
short_negatives = [
    "not good",
    "very bad",
    "movie was bad",
    "not worth watching",
    "boring movie",
]

short_positives = [
    "very good",
    "excellent movie",
    "worth watching",
    "loved it",
    "amazing film",
]

extra_texts = short_negatives + short_positives
extra_labels = [0]*len(short_negatives) + [1]*len(short_positives)

X_train = list(X_train) + extra_texts
y_train = list(y_train) + extra_labels



## 3. Tokenization and Padding

### Step 1: Text → Word Indices (Tokenizer)

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 20000, oov_token = "<OOV>")
tokenizer.fit_on_texts(X_train)

X_tran_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

#### Why num_words=20000?

IMDB reviews have huge vocabulary  
Most sentiment info comes from top frequent words  
Limits noise and memory usage

### Example:
"I loved this movie"
→ [14, 213, 9, 27]

### Step 2: Padding (Very Important for LSTM)

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_tran_seq, maxlen=200, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=200, padding='post')

#### Why maxlen=200?

Most IMDB reviews are long  
But sentiment is usually clear early  
200 is a good tradeoff

In [25]:
print(X_test_pad)

[[   1    1  190 ...  916  696   83]
 [ 238 2057 2057 ...  288 9207 8468]
 [   3  840  121 ... 1006   29    0]
 ...
 [1952  737  737 ...   88   88 3209]
 [1118   61   42 ...   73   73 2246]
 [  24    5    1 ...  243  243    5]]


## 4. Build LSTM Model 

In [26]:
from tensorflow.keras.layers import Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential

# Build the model
model = Sequential()
model.add(Embedding(input_dim=20000, output_dim=64, input_length=200))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2026-02-08 06:32:41.210349: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### Line by line code explanation
* Creates a Sequential model, meaning layers will be stacked one after another in a linear order.
* Converts each word index into a 32-dimensional dense vector, learning word meanings for the top 10,000 words, with each review padded to 200 words.
* Uses an LSTM layer with 32 memory units to learn word order and context, while dropping 20% of inputs and recurrent connections to reduce overfitting.
* Adds a fully connected layer that learns non-linear combinations of LSTM features using the ReLU activation.
* Outputs a single probability value (0–1) using sigmoid activation to classify the review as positive or negative.

#### 🧠 One-Line Summary of the Whole Model

Words are converted into vectors, context is learned using LSTM, features are refined using a dense layer, and final sentiment is predicted using sigmoid.

In [27]:
## Just to see the parameters and layers in our model
model.build(input_shape=(None, 200))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 64)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,354,369 (5.17 MB)

 Trainable params: 1,354,369 (5.17 MB)

 Non-trainable params: 0 (0.00 B)

## 4. Compile the model

In [28]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

#### 🔧 model.compile() — What It Really Does

* compile() tells Keras how to train the model  
– how to measure error,  
– how to update weights,  
– and what to monitor.

#### 1️⃣ Loss Function
loss="binary_crossentropy"
##### What is a loss function?
* A loss function measures how wrong the model’s prediction is compared to the true label.

#### 2️⃣ Optimizer
optimizer="adam"
##### What is an optimizer?
* An optimizer updates the model’s weights to minimize the loss.
* Loss → Gradients → Weight updates

#### 3️⃣ Metrics
metrics=["accuracy"]
##### What is a metric?
* A metric is NOT used for training — it is used for monitoring performance.

Accuracy tells:

“How often is the model correct?”

## 5. Callbacks (training control)

In [29]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Stop training if validation loss does not improve
early_stopping = EarlyStopping(
    monitor="val_loss",        # Watch validation loss
    patience=3,                # Stop after 3 epochs with no improvement
    restore_best_weights=True  # Revert to best model weights
)

# Save the best model during training
model_checkpoint = ModelCheckpoint(
    "best_lstm_model.keras",      # File to save the model
    monitor="val_loss",        # Save based on validation loss
    save_best_only=True,       # Only save the best model
    verbose=1
)

#### EarlyStopping

* EarlyStopping halts training when validation loss stops improving, preventing overfitting and saving computational resources.

#### ModelCheckpoint

* ModelCheckpoint automatically saves the model with the best validation performance, ensuring the most generalizable version is preserved.

## 6. Train the model

In [30]:
y_train = np.array(y_train)

In [31]:
history = model.fit(
    X_train_pad,               # Training input data (padded sequences)
    y_train,                   # Training labels
    epochs=5,                  # Maximum number of epochs
    batch_size=64,             # Number of samples per training step
    validation_split=0.2,      # Validation data
    callbacks=[early_stopping, model_checkpoint],
    verbose=1                  # Show training progress
)

Epoch 1/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.7483 - loss: 0.4683
Epoch 1: val_loss improved from inf to 0.29414, saving model to best_lstm_model.keras
501/501 ━━━━━━━━━━━━━━━━━━━━ 90s 171ms/step - accuracy: 0.7485 - loss: 0.4681 - val_accuracy: 0.8744 - val_loss: 0.2941
Epoch 2/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.9314 - loss: 0.1792
Epoch 2: val_loss did not improve from 0.29414
501/501 ━━━━━━━━━━━━━━━━━━━━ 87s 173ms/step - accuracy: 0.9314 - loss: 0.1793 - val_accuracy: 0.8747 - val_loss: 0.3057
Epoch 3/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.9665 - loss: 0.0997
Epoch 3: val_loss did not improve from 0.29414
501/501 ━━━━━━━━━━━━━━━━━━━━ 83s 166ms/step - accuracy: 0.9665 - loss: 0.0997 - val_accuracy: 0.8697 - val_loss: 0.3874
Epoch 4/5
501/501 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.9835 - loss: 0.0509
Epoch 4: val_loss did not improve from 0.29414
501/501 ━━━━━━━━━━━━━━━━━━━━ 83s 165ms/step - accuracy: 0.9835 - 

#### 📌 What happens during training
* Data flows: Embedding → LSTM → Dense → Output
* Loss is calculated using binary_crossentropy
* Backpropagation updates Embedding, LSTM, and Dense weights
* Training stops early if validation loss stops improving

## 7. Evaluate the model

In [32]:
loss, accuracy = model.evaluate(X_test_pad, np.array(y_test), verbose=0)

print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Loss: 0.2847
Validation Accuracy: 0.8790


## 8. Make predictions (optional)

In [33]:
# Make predictions 
predictions = model.predict(X_test_pad)

# Convert probabilities to class labels
prediction_labels = (predictions > 0.6).astype(int)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step


In [34]:
predictions

array([[0.34771714],
       [0.9075875 ],
       [0.01394469],
       ...,
       [0.86213076],
       [0.06201907],
       [0.33427522]], dtype=float32)

## 9. Build Predictive System

In [35]:
def predict_sentiment(review):
    # Convert text to sequence of word indices
    sequence = tokenizer.texts_to_sequences([review])

    # Pad sequence to match training input length
    padded_sequence = pad_sequences(sequence, maxlen=200, padding='post')

    # Predict sentiment probability
    prediction = model.predict(padded_sequence)

    # Convert probability to label
    sentiment = "Positive" if prediction[0][0] > 0.6 else "Negative"

    return sentiment

## 10. Example Usage

In [36]:
## predict sentiment for a review
new_review = "This movies was fantastic"
customer_sentiment = predict_sentiment(new_review)
print(customer_sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Positive


In [37]:
## predict sentiment for a review
new_review = "The moview was not good"
customer_sentiment = predict_sentiment(new_review)
print(customer_sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Negative


## 11. Save the Model & tokenizer

In [38]:
# Save model
model.save("sentiment_model.keras", include_optimizer=False)

In [39]:
# Save tokenizer
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [ ]:
tokenizer.word_index.get("not_good")